In [1]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.distributions import Categorical
import torchvision
import torchvision.transforms as transforms
import numpy as np
from Simulation import Level

# L1 and L2 are simple and complex examples we will use to test policy
# learning
L1 = ["11111",
     "11011",
     "11111",
     "11111",
     "11X11",
     "11111",
     "11111"]

L2 = [
     "11111",
     "11011",
     "11011",
     "11011",
     "11011",
     "11011",
     "11011",
     "11001",
     "11111",
     "11111",
     "11X11",
     "11111",
     "11111"]

simple_level = Level(L1)
complex_level = Level(L2)

In [2]:
print(simple_level)
print(complex_level)

11111
11011
11111
11111
11X11
11111
11111

11111
11011
11011
11011
11011
11011
11011
11001
11111
11111
11X11
11111
11111



In [3]:
# Hyperparameters
agent_view = 5*5*3
agent_choices = 8
learning_rate = 0.001
gamma = 0.99
hidden_size = 128
dropout_prob = 0.3

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.state_space = agent_view # Input vector
        self.action_space = agent_choices # Number of choices
        
        # Neural Net architecture
        self.l1 = nn.Linear(self.state_space, hidden_size, bias=True)
        self.l2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.l3 = nn.Linear(hidden_size, self.action_space, bias=False)
        
        self.gamma = gamma
        
        # Episode policy and reward history
        self.policy_history = Variable(torch.Tensor())
        self.reward_episode = []
        # Overall reward and loss history
        self.reward_history = []
        self.loss_history = []
    
    def forward(self, x):
        model = torch.nn.Sequential(
            self.l1,
            nn.Dropout(p=dropout_prob),
            nn.SELU(),
            self.l2,
            nn.Dropout(p=dropout_prob),
            nn.SELU(),
            self.l3,
            nn.Softmax(dim=-1)
        )
        return model(x)
        

In [4]:
policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=learning_rate)

In [5]:
# Implement select_action here
def select_action(state):
    state = torch.from_numpy(state).type(torch.FloatTensor)
    choices = policy(Variable(state))
    c = Categorical(choices)
    
    #if(random.random() > 0.1):
    action = c.sample()
    #else:
    #    action = torch.Tensor(1, dtype=tensor.int)
    #    a = int(random.randint(0,8))
    #    action.data[0] = a
    #    print(a)
    #    print(action)
    
        
    if policy.policy_history.nelement() == 0:
        policy.policy_history = torch.stack([c.log_prob(action)])
    else:
        policy.policy_history = torch.cat([policy.policy_history, torch.stack([c.log_prob(action)])])
    
    '''
    if policy.policy_history.size()[0] > 1:
        policy.policy_history = torch.cat([policy.policy_history, c.log_prob(action)])
    elif policy.policy_history.nelement() != 0:
        policy.policy_history = torch.stack([policy.policy_history, c.log_prob(action)])
    else:
        policy.policy_history = (c.log_prob(action))
    '''
    return action
        

In [6]:
# We apply Monte-Carlo Policy Gradient to improve out policy according
# to the equation
def update_policy():
    
    R = 0
    rewards = []

    # Discount future rewards back to the present using gamma
    for r in policy.reward_episode[::-1]:
        R = r + policy.gamma * R
        rewards.insert(0,R)

    # Scale rewards
    rewards = torch.FloatTensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + np.finfo(np.float64).eps)

    # Calculate loss
    loss = (torch.sum(torch.mul(policy.policy_history, Variable(rewards)).mul(-1), -1))

    # Update network weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    policy.loss_history.append(loss.data.item())
    
    #Save and intialize episode history counter
    policy.reward_history.append(np.sum(policy.reward_episode))
    policy.policy_history = Variable(torch.Tensor())
    policy.reward_episode = []

In [7]:
def rfunc0(x,steps,done):
    reward = 0
    if done:
        if(x <= 0):
            reward = 1000
        else:
            reward = -1000
    else:
        reward = 1/(x+2)
    #reward -= steps/4
    return reward


def rfunc1(x,steps,done):
    reward = 20 - (x*3)
    if done:
        if(x <= 0):
            reward += 40 - steps
        else:
            reward -= 40 + steps
    #reward -= steps/4
    return reward

def rfunc2(x,steps,done):
    return 1/(x+2)

def rfunc3(x, steps, done):
    return random.random()

def rfunc4(x, steps, done):
    return 5-x

level = complex_level
max_reward = 1

ActDictionary = {0:"SU",
                 1:"SL",
                 2:"SR",
                 3:"SD",
                 4:"JU",
                 5:"JL",
                 6:"JR",
                 7:"JD"}

def main(episodes):
    for episode in range(episodes):
        done = False     
        level.Reset()
        stps = []
        while not done:
            state = np.asarray(level.getVector())
            action = select_action(state)
            x,steps,done = level.Act(action.item())
            reward = rfunc0(x,steps,done)
            stps += [ActDictionary[action.item()]]
            policy.reward_episode.append(reward)
            if x <= 0:
                print("Reached the end!", end=" ")
            if done:
                break 
        update_policy()
        print("Episode Done!")
        print(stps)

In [8]:
main(1000)

Episode Done!
['SD', 'SL', 'SR', 'JU', 'SR', 'SU', 'JL', 'SR', 'SL', 'JL']
Episode Done!
['SR', 'SU', 'JD', 'JR']
Episode Done!
['JL', 'SD', 'SD', 'JL']
Episode Done!
['JL', 'SR', 'JR', 'SU', 'JL', 'JR', 'JU']
Episode Done!
['SD', 'JD']
Episode Done!
['SR', 'SR', 'SU', 'SD', 'SD', 'SR']
Episode Done!
['SD', 'SR', 'JL', 'SR', 'SU', 'SR', 'JD', 'JU', 'SD', 'JD']
Episode Done!
['SL', 'SU', 'JD', 'SR', 'SR', 'JD']
Episode Done!
['JU', 'SD', 'JR', 'SL', 'JU']
Episode Done!
['SL', 'SU', 'SR', 'JL', 'SR', 'JU', 'SD', 'SU', 'SD', 'SD', 'JR', 'SU', 'SD', 'SU', 'JL', 'SD', 'SR', 'JU']
Episode Done!
['SR', 'SR', 'SR']
Episode Done!
['JL', 'JL']
Episode Done!
['JL', 'SL']
Episode Done!
['SU', 'JL', 'SU', 'SR', 'SR', 'SR', 'JR']
Episode Done!
['SL', 'SR', 'JD', 'SL', 'SU', 'SR', 'JL', 'SR', 'JR', 'SL', 'SD', 'JD']
Episode Done!
['SR', 'SU', 'SD', 'SU', 'SD', 'JU', 'SU']
Episode Done!
['SR', 'JR']
Episode Done!
['JD', 'JU', 'SU', 'SD', 'SU', 'JL', 'SR', 'JR', 'JD', 'SL', 'SR', 'SR', 'JU', 'JR']
Epis

Episode Done!
['SR', 'SD', 'JL', 'JR', 'SD', 'SR', 'SL', 'SU', 'JL', 'SU', 'SR', 'SR', 'JL', 'SR', 'JL', 'SR', 'SU', 'SU', 'JR', 'SL', 'JL', 'SR', 'SR', 'SD', 'JR', 'SU', 'JL', 'SR', 'JL', 'SD']
Episode Done!
['JL', 'SR', 'SU', 'SR', 'JL', 'SR', 'SR', 'SL', 'SU', 'SR', 'SU']
Episode Done!
['JL', 'JR', 'SU', 'JL', 'SR', 'SU', 'SD', 'SD', 'SU', 'JR', 'SD', 'JU', 'JL', 'SR', 'SL', 'SD', 'SU', 'JR', 'JL', 'JR', 'SR', 'SL', 'SR', 'JL', 'SL', 'SD', 'SR', 'SU', 'SR', 'SU']
Episode Done!
['SD', 'SR', 'SR', 'SL', 'JL', 'SU', 'SU', 'SR', 'SD', 'SR', 'SL', 'JR', 'JL', 'SL', 'SR', 'SL', 'JU', 'SU', 'SD', 'JR', 'SD', 'JD', 'JL', 'SR', 'SL', 'SU', 'SD', 'JD']
Episode Done!
['JL', 'SR', 'SU', 'JR', 'JD', 'JL', 'SR', 'SL', 'SU', 'JD', 'JD']
Episode Done!
['JL', 'SR', 'SR', 'SR', 'SU', 'JL', 'SU', 'JR', 'SL', 'SD', 'SD', 'SL', 'SU', 'SD', 'JR', 'JL', 'JU', 'JD', 'JD', 'SR', 'SU', 'SR', 'SL', 'JL', 'JR', 'SU', 'JR', 'JL', 'SD', 'SR']
Episode Done!
['JD', 'JR', 'SL', 'SR', 'SL', 'SR', 'SU', 'SL', 'JL', '

Episode Done!
['JR', 'JL', 'JL', 'SR', 'SR', 'JD', 'SR', 'JU', 'JL', 'SU', 'SR', 'SD', 'JR', 'JL', 'SL', 'SR', 'SD', 'SR', 'SR', 'JL', 'SU', 'SR', 'SL', 'JR', 'SL', 'JL', 'SR', 'JD', 'SL', 'SR']
Episode Done!
['SD', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'JL', 'SR', 'SR', 'JL', 'SR', 'SR', 'SL', 'JU', 'SR', 'SL', 'SU', 'JU', 'SD', 'JR']
Episode Done!
['SL', 'SR', 'SR', 'SL', 'JU', 'JD', 'JD', 'JR', 'SL', 'SR', 'JL', 'SR', 'SL', 'SR', 'SR', 'JL', 'SR', 'SR', 'SL', 'SR', 'JL', 'SR', 'JL', 'SU', 'SR', 'JL', 'SR', 'SR', 'SR', 'SD']
Episode Done!
['SL', 'SR', 'JL', 'SR', 'SR', 'SD', 'SR', 'JD']
Episode Done!
['JD', 'SR', 'SR', 'SL', 'SR', 'JL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'SL', 'SL', 'SR', 'SR', 'SR', 'SL', 'SL', 'SR', 'JL', 'SR', 'SR', 'SL', 'SR', 'SR', 'SL', 'SR']
Episode Done!
['SD', 'SR', 'JL', 'SR', 'SL', 'SR', 'SR', 'JL', 'SR', 'SL', 'SR', 'SR', 'JL', 'SR', 'SU', 'JD', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'JR']
Episode Done!
['SL', 'SR', '

Episode Done!
['JR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'JL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SL']
Episode Done!
['SR', 'SR', 'SL', 'SL', 'SR', 'SR', 'JL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL']
Episode Done!
['SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'JL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL']
Episode Done!
['SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SL']
Episode Done!
['SR', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'JL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SR', 'JL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR']
Episode Done!
['SR', 'SL'

Episode Done!
['SR', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'JL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'JR']
Episode Done!
['SR', 'SR', 'SL', 'SR', 'JL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL']
Episode Done!
['SR', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'JL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'JL', 'SR', 'SR', 'SL', 'SL']
Episode Done!
['SR', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SR']
Episode Done!
['SR', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'JL', 'SR', 'SR', 'SR', 'SR', 'SL', 'SR', 'JL', 'SR', 'SL']
Episode Done!
['SR', 'SR', 'SL', 'SR', 'JL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SR'

...

Episode Done!
['SL', 'SR', 'SL', 'SR', 'SL', 'JR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'SU', 'SD', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'JR', 'SL', 'SL', 'SL', 'SR', 'SR', 'SL']
Episode Done!
['SL', 'SU', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SU', 'SL', 'SR', 'SD', 'SD', 'SR', 'SL', 'SR', 'SL', 'SR', 'SU', 'SL', 'SR', 'SL']
Episode Done!
['SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SU', 'SR', 'SL', 'SU', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'JR', 'SL', 'SL', 'SL', 'SR', 'SL', 'SR', 'SD', 'SL']
Episode Done!
['SL', 'SR', 'SL', 'SR', 'SL', 'JU', 'SR', 'SL', 'JR', 'SL', 'SL', 'SL', 'SR', 'SU', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR']
Episode Done!
['SL', 'JR', 'SL', 'SL', 'SR', 'SL', 'SU', 'SU', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'JU', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL']
Episode Done!
['SL', 'SR'

Episode Done!
['SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SU', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SU', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR']
Episode Done!
['SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SU', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL']
Episode Done!
['SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SU', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'JL', 'SR', 'SR', 'SL', 'SR']
Episode Done!
['SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'SL', 'SR', 'SR', 'SL', 'SU', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SL']
Episode Done!
['SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR']
Episode Done!
['SL', 'SR'

Episode Done!
['SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SU', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'SL']
Episode Done!
['SL', 'SR', 'SL', 'SR', 'SL', 'JD', 'SR', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SU', 'SR', 'SL']
Episode Done!
['SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SD', 'SU', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL']
Episode Done!
['SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'JR', 'SL', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL']
Episode Done!
['SL', 'SD', 'SR', 'SL', 'SR', 'SL', 'SR', 'SR', 'SL', 'SU', 'SL', 'SR', 'SL', 'SR', 'SU', 'SL', 'JR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL']
Episode Done!
['SL', 'SU'

Episode Done!
['SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SD', 'SL', 'SR', 'SU', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR']
Episode Done!
['SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SD', 'SU', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SD', 'SL', 'JR', 'SL', 'SL', 'SR', 'SR', 'SL', 'SR', 'SL', 'SL', 'SR', 'SL', 'SR', 'SL']
Episode Done!
['SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SU', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL']
Episode Done!
['SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR']
Episode Done!
['SL', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'SR', 'SL', 'SR', 'SL', 'SR', 'SD', 'SL', 'SU', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SR', 'SL', 'SU', 'SD', 'SR', 'SL', 'SR']
Episode Done!
['SL', 'SL'

KeyboardInterrupt: 